In [1]:
import csv
from itertools import islice
import pickle
import time

import pandas as pd
import tqdm
from models.dummy import DummyModel
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import ExtraTreesClassifier

/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PREDICTION_FILE_HEADERS = 'TransactionID,isFraud\n'
FEATURE_COLUMNS = [ # quick and dirty
#   'TransactionID',
#   'isFraud',
  'TransactionDT',
  'TransactionAmt',
#   'ProductCD',
  'card1',
  'card2',
  'card3',
#   'card4',
  'card5',
#   'card6',
  'addr1',
  'addr2',
  'dist1',
  'dist2',
#   'P_emaildomain',
#   'R_emaildomain',
  'C1',
  'C2',
  'C3',
  'C4',
  'C5',
  'C6',
  'C7',
  'C8',
  'C9',
  'C10',
  'C11',
  'C12',
  'C13',
  'C14',
  'D1',
  'D2',
  'D3',
  'D4',
  'D5',
  'D6',
  'D7',
  'D8',
  'D9',
  'D10',
  'D11',
  'D12',
  'D13',
  'D14',
  'D15',
  'M1',
  'M2',
  'M3',
  'M4',
  'M5',
  'M6',
  'M7',
  'M8',
  'M9'
  # ignore their features for now
]

In [3]:
with open('./data/train_transaction_clean.pkl', 'rb') as train_file:
    transactions_train = pickle.load(train_file)

with open('./data/test_transaction_clean.pkl', 'rb') as test_file:
    transactions_test = pickle.load(test_file)

In [4]:
def features(cleaned_transactions):
    features = cleaned_transactions.loc[:, FEATURE_COLUMNS]
    return features.fillna(features.median())

In [5]:
def write_predictions(
    transaction_ids,
    predictions,
    filename='predictions'
):
    if len(transaction_ids) != len(predictions):
        raise TypeError("labels don't match predictions")
    with open(f'./data/submissions/{filename}.csv', 'w') as predictions_file:
        predictions_file.write(PREDICTION_FILE_HEADERS)
        for transaction_id, prediction in zip(transaction_ids, predictions):
            predictions_file.write(f'{transaction_id},{prediction}\n')

In [6]:
train_features = features(transactions_train)
train_targets = transactions_train.isFraud
del transactions_train

In [7]:
scaler = RobustScaler().fit(train_features)

In [8]:
etc = ExtraTreesClassifier(
    bootstrap=True,
    criterion='entropy',
    max_features=0.05,
    min_samples_leaf=13,
    min_samples_split=9,
    n_estimators=100
)

In [9]:
etc.fit(scaler.transform(train_features), train_targets)

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
           max_depth=None, max_features=0.05, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=13, min_samples_split=9,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [10]:
test_features = features(transactions_test)
predictions = etc.predict(test_features)
write_predictions(test_features.index, predictions, 'extra_trees_1_79686591')